### 【 데이터 전용 Dataset/DataLoader 】
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset    : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치크기만큼 데이터를 추출해 주는 역할
- 데이터셋 분리 => random_split() 함수 제공 : 타겟 클래스 고려하지 않은 랜덤한 데이터 분리

[1] 모듈 로딩 및 데이터 준비 <hr>

In [17]:
# [1-1] 모듈 로딩
import torch                                        # 텐서 및 수치과학 함수들 관련 모듈
from torch.utils.data import Dataset, DataLoader    # pytorch의 데이터 로딩
from torch.utils.data import random_split           # pytorch의 데이터셋 분리 함수
import pandas as pd

In [18]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

# [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE)
testDF  = pd.read_csv(TEST_FILE)

[2] 커스텀 데이터셋 클래스 생성 및 데이터 적용 <hr>

In [19]:
# -------------------------------------------------------------------------------------
# [2-2] 커스텀 데이터셋 클래스 정의
# -------------------------------------------------------------------------------------
# 클래스이름 : ClfDataset
# 부모클래스 : Dataset
# 오버라이딩 : _ _init_ _(self)         : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
#            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
#            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
#                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
# -------------------------------------------------------------------------------------
class ClfDataset(Dataset):

    #- 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    #- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0]
    
    #- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS
    

In [20]:
# -------------------------------------------------------------------------------------
# [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
# -------------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)   ## <= trainDS, validDS 분리
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 10000,  testDS : 2000


In [21]:
# -------------------------------------------------------------------------------------
# [2-4] 학습용/검증용/테스트용 데이터셋 분리
# -------------------------------------------------------------------------------------
# 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
# 검증용   : 제대로 데이터셋에서 규칙/패턴을 찾는지 확인 용도
#           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
# 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

# 학습용 데이터셋의 개수
print(f'allDS   :{len(allDS)}개, testDS : {len(testDS)}개')

# 학습용 데이터셋 => 학습용:검증용 = 80:20
TRAIN_SIZE = int(0.8 * len(allDS))
VALID_SIZE = len(allDS) - TRAIN_SIZE

print(f'trainDS :{TRAIN_SIZE}개')
print(f'validDS :{VALID_SIZE}개')
print(f'testDS  :{len(testDS)}개')

allDS   :10000개, testDS : 2000개
trainDS :8000개
validDS :2000개
testDS  :2000개


In [22]:
# 학습용 데이터셋 분리 => random_split()
# 단점) 분류의 경우 타겟의 비율 고려되지 않음!
genSeed = torch.Generator().manual_seed(10)
trainDS, validDS = random_split(allDS,
                                [TRAIN_SIZE, VALID_SIZE],
                                generator=genSeed)

print(f'trainDS :{type(trainDS)}, {len(trainDS)}개')
print(f'validDS :{type(validDS)}, {len(validDS)}개')
print(f'testDS  :{type(testDS)}, {len(testDS)}개')

trainDS :<class 'torch.utils.data.dataset.Subset'>, 8000개
validDS :<class 'torch.utils.data.dataset.Subset'>, 2000개
testDS  :<class '__main__.ClfDataset'>, 2000개


In [23]:
# -------------------------------------------------------------------------------------
# [2-2-3] 학습용/검증용/테스트용 데이터셋 속성
# -------------------------------------------------------------------------------------
# - datasets 타입 속성
print(f"type(testDS)---------------")
print(testDS.y.shape, len(testDS.y), sep='\n')

# - dataset.Subset타입 속성
# - dataset                 속성 : 쪼개지기 전의 원본 데이터셋 정보 확인
# - indices                 속성 : 선택된 데이터의 인덱스 정보
# - dataset.data            속성 : 실제 이미지의 로우 데이터 즉, ndarray
print(f"\n{type(trainDS)}----------------")
print(trainDS.dataset,
      trainDS.indices,
      trainDS.dataset.y.shape,
      len(trainDS.indices),
      sep='\n')

print(f"\n{type(validDS)}----------------")
print(validDS.dataset,
      validDS.indices,
      validDS.dataset.y.shape,
      len(validDS.indices),
      sep='\n')

type(testDS)---------------
(2000,)
2000

<class 'torch.utils.data.dataset.Subset'>----------------
[6937, 7735, 6762, 5034, 7468, 6550, 8999, 4970, 5553, 4654, 726, 837, 5561, 210, 3992, 1313, 9488, 3580, 814, 7512, 6988, 8415, 1658, 3531, 8479, 4059, 3548, 6117, 431, 1603, 5554, 6426, 7048, 7433, 4950, 5244, 1659, 3200, 6631, 9051, 8597, 2300, 1145, 8740, 2901, 3586, 2188, 179, 1774, 7052, 3166, 8011, 8223, 2154, 7724, 9416, 5994, 5723, 2395, 9710, 3059, 9948, 4992, 9741, 4908, 9309, 8020, 4696, 9975, 1780, 1182, 4021, 9503, 9260, 3359, 7861, 730, 5599, 8861, 9968, 1329, 721, 3736, 4942, 1624, 1461, 7807, 9987, 5583, 4332, 2830, 6949, 8976, 422, 3836, 6602, 3391, 5160, 2328, 3176, 9585, 9640, 5880, 9301, 4306, 7523, 8161, 3788, 1012, 6450, 6344, 1249, 5670, 1167, 2693, 5186, 2516, 4590, 5662, 1700, 4025, 1810, 4331, 2659, 5685, 3750, 2927, 9938, 1541, 2972, 9756, 8725, 9551, 6462, 9152, 7642, 2898, 423, 3064, 683, 483, 875, 6331, 9113, 2757, 1917, 2266, 9320, 6194, 3793, 678, 4846, 7

In [24]:
# -------------------------------------------------------------------------------------
# 학습용/검증용/테스트용 데이터셋에 카테고리별 데이터 분포
# - 균형 데이터셋 & 불균형 데이터셋
# -------------------------------------------------------------------------------------
from collections import Counter

# 테스트 데이터셋의 카테고리별 분포
testC = Counter(testDS.y.tolist())
print(f"testDS -> \n{dict(sorted(testC.items()))}")

# 학습용 데이터셋의 카테고리별 분포
sel_train = [trainDS.dataset.y[idx].item() for idx in trainDS.indices]
trainC = Counter(sel_train)

trainDict = dict(sorted(trainC.items()))
print(f"trainDict -> \n{trainDict}")\

# 검증용 데이터셋의 카테고리별 분포
val_train = [validDS.dataset.y[idx].item() for idx in validDS.indices]
validC = Counter(val_train)

validDict = dict(sorted(validC.items()))
print(f"validDict ->\n{validDict}")

testDS -> 
{0: 175, 1: 234, 2: 219, 3: 207, 4: 217, 5: 179, 6: 179, 7: 204, 8: 192, 9: 194}
trainDict -> 
{0: 795, 1: 884, 2: 794, 3: 833, 4: 769, 5: 672, 6: 814, 7: 886, 8: 758, 9: 795}
validDict ->
{0: 206, 1: 243, 2: 197, 3: 200, 4: 211, 5: 190, 6: 200, 7: 184, 8: 186, 9: 183}
